In [13]:
from torch import nn
import torch
import math
import torch.nn.functional as F
import torch.optim as optim

dtype  = torch.float32
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
torch.set_default_device(device)


##### Attention:
For this lab, there are no local answers for the questions in learntools. However, there are some assertions for the first 4 questions. You should pass all of the assertions before submitting to the autograding.

Some of the questions use mathematical symbols and equations to depict the scenario. These are just some basic "formal" definitions and you should understand them. In case you haven't understood them, read again and give yourself sometime to ponder. If there is any error or details in the questions that heavily confuse you, please post your concerns on Google Classroom. 

## Questions

### Q1: Given a function $f(x) = x^2 + 3x + 8$, return the gradient of $x$ when $x=2.0$ using pytorch autograd. The gradient tensor should have type torch.float32 and dim=0.

In [14]:
def exercise_1() -> torch.Tensor:
    #Create a tensor with requires_grad=True
    # and set it to 2.0
    x = torch.tensor(2.0,  requires_grad=True)

    # Compute the function f(x) = x^2 + 3x + 8
    f_x = x**2 + 3 * x + 8

    # Compute the gradient of f(x) with respect to x and then return it
    # It should be 2x + 3. When x=2, the gradient should be 7
    f_x.backward()
    return x.grad


ex1_sol = exercise_1()
assert torch.equal(ex1_sol, torch.tensor(7.0))

### Q2: A typical layer in a Neural Network would look like $f(x) = w_1 x_1 + w_2 x_2 + w_3 x_3 + ... + b$, where $w_j$ is the j-th weight of the function and $b$ is its bias. This function outputs a result for a multi-dimension input, where $x_j$ is the j-th feature of the input. Implement this function, return the gradients of each weight for a given input `x` and the output using `tuple` with the formar `(output, weight_grads_tensor)`. The weights should be randomized using `torch.randn()` and the bias must be set to $5.0$. The input used by the autograder will be the same size as the one given below.

In [15]:
x = torch.tensor([1, 2, 3, 4, 5])

def exercise_2(x) -> tuple:

    # Set Bias to 5.0
    bias = torch.tensor(5.0, dtype=dtype, requires_grad=True)

    # Create random Tensors for weights.
    torch.manual_seed(0) # must set seed=0 before random your weight
    a = torch.randn((), dtype=dtype, requires_grad=True)
    b = torch.randn((), dtype=dtype, requires_grad=True)
    c = torch.randn((), dtype=dtype, requires_grad=True)
    d = torch.randn((), dtype=dtype, requires_grad=True)
    e = torch.randn((), dtype=dtype, requires_grad=True)

    # Compute the function f(x) = w1*x1 + w2*x2 + w3*x3 + w4*x4 + w5*x5 + bias
    output =  a * x[0] + b * x[1] + c * x[2] + d * x[3] + e * x[4] + bias

    # Backprop
    output.backward()

    # you must return in this format
    # return (output, weight_grads_tensor)
    return output, torch.tensor([a.grad, b.grad, c.grad, d.grad, e.grad], dtype=dtype)

ex2_sol = exercise_2(x)

assert (ex2_sol[0] - torch.tensor(-3.7311) < 1e-4).item()
assert torch.equal(ex2_sol[1], torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32))

### Q3: Suppose each feature $x_j$ needs a separate function $f_j(x_j)$, where $f_j(x_j) = w_j x_j + b_j$. In other words, each feature requires a function, which has a separate set of 1 weight and 1 bias. Therefore, the output $y$ of your function $G(x)=[f_1(x_1), f_2(x_2), ..., f_n(x_n)]$ should be $y = [(w_1 x_1 + b_1), (w_2 x_2 + b_2), ..., (w_n x_n + b_n)]$ (**note:** these are matrices written lazily with markdown). Implement the function $G(x)$, return the output and the gradient sets for each function for a given `x`. The weights should be randomized using `torch.randn()`, and the bias is the same as $j$ of $f_j()$ (for example: $b_1 = 1.0$, $b_2 = 2.0$). The input used by the autograder will be the same size as the one given below.

In [16]:
x = torch.tensor([1, 2, 3, 4, 5])

def exercise_3(x) -> tuple:
     # all the output and gradients are tensors
    # Set Biases
    b1 = torch.tensor(1.0, requires_grad=True)
    b2 = torch.tensor(2.0, requires_grad=True)
    b3 = torch.tensor(3.0, requires_grad=True)
    b4 = torch.tensor(4.0, requires_grad=True)
    b5 = torch.tensor(5.0, requires_grad=True)
    
    # Create random Tensors for weights.
    torch.manual_seed(0) # must set seed=0 before random your weight
    a = torch.randn((), dtype=dtype, requires_grad=True)
    b = torch.randn((), dtype=dtype, requires_grad=True)
    c = torch.randn((), dtype=dtype, requires_grad=True)
    d = torch.randn((), dtype=dtype, requires_grad=True)
    e = torch.randn((), dtype=dtype, requires_grad=True)

    # Compute the function f(x) 
    y1 = a * x[0] + b1
    y2 = b * x[1] + b2
    y3 = c * x[2] + b3
    y4 = d * x[3] + b4
    y5 = e * x[4] + b5

    # Backprop
    y1.backward()
    y2.backward()
    y3.backward()
    y4.backward()
    y5.backward()

    # you must return in this format
    # return (output, weight_grads_tensor)
    output = torch.tensor([y1.item(), y2.item(), y3.item(), y4.item(), y5.item()], dtype=dtype)
    weight_grads = torch.tensor([a.grad, b.grad, c.grad, d.grad, e.grad], dtype=dtype)

    return output, weight_grads

ex3_sol = exercise_3(x)
assert torch.all((ex3_sol[0] - torch.tensor([ 2.5410,  1.4131, -3.5364,  6.2737, -0.4226 ], dtype=torch.float32)) < 5e-4).item()
assert torch.equal(ex3_sol[1], torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32))
# q3.check()

### Q4: Using the function $f(x)$ similar to the one in question 2, implement the function with random weights and bias (must use `torch.manual_seed(0)` before each initialization). Calculate the Mean Squared Error between the `target` and your function prediction on the `data` as a loss metric, then return a tuple containing the gradients of your function weights and bias based on the calculated loss.

In [17]:
data = torch.tensor([2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0])
target = torch.tensor([2.0])

def exercise_4(data, target) -> tuple:
    # initialize weights with seed=0
    torch.manual_seed(0)
    # Create random Tensors for weights.
    a = torch.randn((), dtype=dtype, requires_grad=True)
    b = torch.randn((), dtype=dtype, requires_grad=True)
    c = torch.randn((), dtype=dtype, requires_grad=True)
    d = torch.randn((), dtype=dtype, requires_grad=True)
    e = torch.randn((), dtype=dtype, requires_grad=True)
    f = torch.randn((), dtype=dtype, requires_grad=True)
    g = torch.randn((), dtype=dtype, requires_grad=True)
    h = torch.randn((), dtype=dtype, requires_grad=True)
    i = torch.randn((), dtype=dtype, requires_grad=True)
    j = torch.randn((), dtype=dtype, requires_grad=True)

    # Set Bias
    torch.manual_seed(0)
    bias = torch.randn((), requires_grad=True)

    # Compute the function f(x)
    y = ( a*data[0] + b*data[1] + c*data[2] + d*data[3] + e*data[4] + f*data[5] + g*data[6] + h*data[7] + i*data[8] + j*data[9] + bias)

    # Compute the MSE loss
    loss = (y - target) ** 2

    # Backprop
    loss.backward()

    # gather grads into 1‑D tensors
    weight_grads = torch.stack([a.grad, b.grad, c.grad, d.grad, e.grad, f.grad, g.grad, h.grad, i.grad, j.grad], dim=0) ## shape (10,)
    bias_grads   = torch.tensor([bias.grad], dtype=dtype) # shape (1,)

    # weight_grads = torch.tensor([a.grad, b.grad, c.grad, d.grad, e.grad, f.grad, g.grad, h.grad, i.grad, j.grad], dtype=dtype)#
    return weight_grads, bias_grads

ex4_sol = exercise_4(data, target)
assert ex4_sol[0].dim() == 1 and ex4_sol[1].dim() == 1, print("All tensors must have dim == 1")
assert torch.all((ex4_sol[0] - torch.tensor([-85.1518, -127.7277, -170.3036, -212.8795, -255.4553, -298.0312, -340.6071, -383.1830, -425.7589, -468.3348], dtype=torch.float32)) < 5e-4)
assert torch.all((ex4_sol[1] - torch.tensor([-42.5759], dtype=torch.float32)) < 5e-4)

### Q5: This question worth 40% the points of this lab. Your solution for this question will be graded manually by the TAs, hence you will be informed later for the total points for this lab after the deadline has been met.

**Note**: This question will not focus on getting the right results as your implementation is more important. Therefore there will be no local answer or autograding.

#### Create a small neural network consisting 2 Linear layers, each followed by a Sigmoid activation function. The template code has been provided below including the return format (`x` should be replaced by the final `tensor` after the forward pass). Your code must use layers and functions provided by `pytorch`. Initialize all necessary components that can be used during the training phase including a Binary Cross Entropy Loss function and a Gradient Descent optimizer. Train your model for 2 epochs with a proper train loop, then make a prediction on a given `val_x` and return your prediction.

**Note**: the first Linear Layer can have any `in_features` and `out_features`, but the last Linear Layer must have `out_features=2`. Your model will consume the whole input for each epoch, so you don't need to separate the input into batches.

*Context*: This part is not necessary for your implementation, but it could help you to understand the work that your network is simulating. The `train_x` consists grades for 5 subjects of 20 students, with the minimum grade is 1 and the maximum is 10. The problem is a classification task, when your model predicts if a student is good or bad based on their 5 grades. That is the reason why the final layer must have 2 output features, which correspond to the 2 classes "good" and "bad".

In [ ]:
# train_x = torch.randint(1, 10, (20, 5), dtype=torch.float32) # input
# train_y = (train_x.sum(1) > 30).long() # target

# val_x = torch.randint(3, 10, (10, 5), dtype=torch.float32)

learning_rate = 0.01

"""
    A typical training procedure for a neural network is as follows:
    1. Define the neural network that has some learnable parameters (or weights)
    2. Iterate over a dataset of inputs
    3. For each input, compute the output of the network
    4. Compare the output to the target using a loss function
    5. Compute the loss (how far is the output from being correct)
    6. Propagate gradients back into the network's parameters
    7. Update the weights of the network using a learning rate
"""

def exercise_5(train_x: torch.Tensor,
               train_y: torch.Tensor,
               val_x: torch.Tensor) -> torch.Tensor:
    """
    Builds and trains a 2-layer neural network on (train_x, train_y) for 2 epochs,
    then returns the predicted class labels (0 or 1) for val_x.
    """
    # Define the network
    class MyNetwork(nn.Module):
        def __init__(self, in_features: int = 5, hidden_size: int = 8):
            super(MyNetwork, self).__init__()
            # First layer: in_features (for the 5 subject grades) -> hidden_size
            self.linear1 = nn.Linear(in_features, hidden_size)  
            self.sigmoid1 = nn.Sigmoid()

            # Second layer: hidden_size -> 2 output classes (good/bad)
            self.linear2 = nn.Linear(hidden_size, 2)  
            self.sigmoid2 = nn.Sigmoid()

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            # Layer 1
            x = self.linear1(x)              # shape: (N,8)
            x = self.sigmoid1(x)             # shape: (N,8)

            # Layer 2
            x = self.linear2(x)              # shape: (N,2)
            x = self.sigmoid2(x)             # shape: (N,2)

            # Final output
            return x
    
    # Create an instance of the network
    model = MyNetwork()

    # # This for illustration only
    # params = list(model.parameters())
    # print(f"No of parameter tensors: {len(params)}")

    # # Input example - 3 students with 5 grades 
    # input_example = torch.randn(3, 5) # three students with five grades
    # output_example = model(input_example) # three students with two outputs
    # print(f"\nInput shape: {input_example.size()}")
    # print(f"Output shape: {output_example.size()}")

    # # Create a dummy target (3 students, 2 classes)
    # target = torch.tensor([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]])  # 3 students with binary classification
    # print(f"\nTarget shape: {target.size()}")

    # Optimizer
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    #This is for the BCELoss function because it compares each output probability to a target probability (0 or 1) for that class—so you need a target vector of the same shape
    train_y_one_hot = torch.zeros(train_y.size(0), 2)
    train_y_one_hot.scatter_(1, train_y.unsqueeze(1), 1.0)
    # Example: train_y = tensor[0, 1, 1, 0]
    # train_y_one_hot = torch.zeros(4, 2)        # [[0,0], [0,0], [0,0], [0,0]]
    # train_y_one_hot.scatter_(1, train_y.unsqueeze(1), 1.0)
    # # → [[1,0],
    # #    [0,1],
    # #    [0,1],
    # #    [1,0]]
    

    # Loss function - Binary Cross Entropy
    criterion = nn.BCELoss()

    # Training loop: 2 epochs
    for i in range(2):
        #foward pass
        output = model(train_x)            # shape: (N,2)
        # Compute the loss
        loss = criterion(output, train_y_one_hot) # shape: (N,2)
        # Backward pass
        optimizer.zero_grad()              # gradient buffers had to be manually set to zero using optimizer.zero_grad(). This is because gradients are accumulatedas
        loss.backward()                    # compute gradients
        optimizer.step()                   # Does the update

    with torch.no_grad():
        predictions = model(val_x)          


    return predictions

# Test the function

# ----------- TESTING SCRIPT -----------
if __name__ == "__main__":
    # Generate random dataset
    torch.manual_seed(0)
    train_x = torch.randint(1, 11, (20, 5), dtype=torch.float32)
    train_y = (train_x.sum(1) > 30).long()
    val_x = torch.randint(3, 10, (10, 5), dtype=torch.float32)

    # Run exercise_5
    predictions = exercise_5(train_x, train_y, val_x)

    # Print results
    print("Train labels:    ", train_y.tolist())
    print("Validation data:")
    print(val_x)
    print("Predictions:     ", predictions.tolist())


Train labels:     [0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]
Validation data:
tensor([[6., 8., 3., 7., 5.],
        [9., 4., 8., 5., 6.],
        [7., 5., 9., 4., 5.],
        [7., 6., 7., 5., 6.],
        [7., 7., 4., 9., 3.],
        [8., 4., 8., 3., 4.],
        [3., 7., 9., 5., 3.],
        [5., 7., 9., 5., 7.],
        [3., 4., 8., 9., 6.],
        [6., 8., 7., 9., 5.]])
Predictions:      [[0.37289589643478394, 0.5060998797416687], [0.392078161239624, 0.5023755431175232], [0.4019102454185486, 0.5087367296218872], [0.39243084192276, 0.5058785080909729], [0.3403412401676178, 0.5028472542762756], [0.40766486525535583, 0.5054919719696045], [0.3813575804233551, 0.5153592228889465], [0.40386247634887695, 0.5113861560821533], [0.34043651819229126, 0.4929261803627014], [0.34823328256607056, 0.5054110884666443]]
